# Processing the Query

In [126]:
import psycopg2
import pandas as pd
from tqdm import tqdm
import json
import os
import chardet
import requests
import streamlit as st
from llama_index import StorageContext, load_index_from_storage


openai_key = ''

# Set OpenAI API Key
os.environ['OPENAI_API_KEY'] = openai_key

# Specify the storage directory
persist_dir = "/Users/tanmay/Documents/Pro-Projects/CDSS-MIMIC/VectorStore_Indexes"

# Initialize the storage context with the persist_dir
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

# Load the index from the storage context
index = load_index_from_storage(storage_context)

print("Index successfully loaded from storage.")

# Create a query engine from the index
query_engine = index.as_query_engine()

print("Successfully createa a query engine")


def GPT_Input_Preprocessor(user_input):

    KEY = openai_key
    URL = "https://api.openai.com/v1/chat/completions"

    context_text = """
    You are an expert in structuring patient data for a Clinical Decision Support System (CDSS).
    Your task is to extract patient details and organize them into a structured JSON format.
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {KEY}"
    }

    messages = [
        {"role": "system", "content": context_text},
        {"role": "user", "content": str(user_input)}
    ]

    payload = {
        "model": "gpt-4",
        "messages": messages,
        "temperature": 0
    }

    response = requests.post(URL, headers=headers, json=payload)
    if response.status_code == 200:
        return json.loads(response.json()["choices"][0]["message"]["content"])
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


def Generate_JSON_Output(User_Query):

  Instructions1 = "Return all possible information for what is requested along with the information about the subject such as subject_id as Patient ID, gender and age. Make sure you give separate output for different subject_ids."

  CDSS_Query1 = User_Query + Instructions1
  response = query_engine.query(CDSS_Query1)

  patients_data = []
  patient_json = GPT_Input_Preprocessor(response)
  if patient_json:
    patients_data.append(patient_json)

  return patients_data


Index successfully loaded from storage.
Successfully createa a query engine


In [129]:
User_Query = "Which patients had abnormal liver function test results and were diagnosed with hepatic cirrhosis or portal hypertension?"

output = Generate_JSON_Output(User_Query)

In [130]:
output

[[{'Patient ID': 21636229,
   'Gender': 'M',
   'Age': 57,
   'Abnormal Liver Function Test Results': {'Bilirubin, Total': '1.9 mg/dL',
    'Hematocrit': '29.5%'},
   'Diagnoses': ['Alcoholic cirrhosis of liver with ascites',
    'Portal hypertension']},
  {'Patient ID': 20429160,
   'Gender': 'M',
   'Age': 43,
   'Abnormal Liver Function Test Results': {'PT': '16.9 sec',
    'Hemoglobin': '9.4 g/dL'},
   'Diagnoses': ['Abscess of liver', 'Acute and subacute necrosis of liver']}]]

# Frontend UI

In [124]:

# Step 1: Delete the file if it exists
if os.path.exists('plan3.py'):
    os.remove('plan3.py')

# Step 2: Write the Streamlit app code to app.py
with open('plan3.py', 'w') as f:
    f.write('''
            
import psycopg2
import base64
import pandas as pd
from tqdm import tqdm
import json
import os
import requests
import streamlit as st
from llama_index import StorageContext, load_index_from_storage

# Set OpenAI API Key
openai_key = ''
os.environ['OPENAI_API_KEY'] = openai_key

# Specify the storage directory
persist_dir = "/Users/tanmay/Documents/Pro-Projects/CDSS-MIMIC/VectorStore_Indexes"

# Initialize the storage context with the persist_dir
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

# Load the index from the storage context
index = load_index_from_storage(storage_context)

# Create a query engine from the index
query_engine = index.as_query_engine()

def GPT_Input_Preprocessor(raw_response):
    """
    Sends the user's input to OpenAI GPT API to preprocess the input.
    """
    URL = "https://api.openai.com/v1/chat/completions"

    context_text = """
    You are an expert in structuring patient data for a Clinical Decision Support System (CDSS).
    Your task is to extract patient details and organize them into a structured JSON format.
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_key}"
    }

    messages = [
        {"role": "system", "content": context_text},
        {"role": "user", "content": str(raw_response)}
    ]

    payload = {
        "model": "gpt-4",
        "messages": messages,
        "temperature": 0
    }

    response = requests.post(URL, headers=headers, json=payload)
    if response.status_code == 200:
        return json.loads(response.json()["choices"][0]["message"]["content"])
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

def Generate_JSON_Output(user_query):
    """
    Generates a JSON output based on the user's query and OpenAI API response.
    """
    instructions = "Return all possible information for what is requested along with the information about the subject such as subject_id as Patient ID, gender, and age. Make sure you give separate output for different subject_ids."
    cdss_query = user_query + instructions

    # Query the index
    response = query_engine.query(cdss_query)

    # Process the response into structured JSON
    if response:
        patients_data = []
        patient_json = GPT_Input_Preprocessor(response.response)  # Ensure `.response` is accessed properly
        if patient_json:
            patients_data.append(patient_json)

        return patients_data
    else:
        st.error("Failed to retrieve data from the query engine.")
        return None


                     
# Streamlit UI for Sidebar Navigation

st.set_page_config(layout="wide", page_title="Clinical Decision Support System (CDSS)")

# Encode the image into base64
image_path = "/Users/tanmay/Documents/Pro-Projects/CDSS-MIMIC/image-med.png"
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Add CSS for sidebar background
sidebar_background = f"""
<style>
    [data-testid="stSidebar"] {{
        background-image: url("data:image/jpeg;base64,{encoded_image}");
        background-size: cover;
        background-repeat: no-repeat;
        background-position: center;
        opacity: 1.0; /* Adjust this value to change opacity */
    }}
</style>
"""

# Inject CSS into Streamlit
st.markdown(sidebar_background, unsafe_allow_html=True)

st.title("🏥 Clinical Decision Support System (CDSS)")

# Sidebar for query and patient navigation
with st.sidebar:
    st.header("Navigation")
    user_query = st.text_area("Enter your query:", height=150)

    if st.button("Submit Query"):
        if user_query.strip():
            try:
                # Ensure the output is a list of patient dictionaries
                raw_output = Generate_JSON_Output(user_query)
                patients_data = raw_output[0] if isinstance(raw_output, list) and isinstance(raw_output[0], list) else raw_output
                st.session_state['patients_data'] = patients_data
                st.session_state['current_patient_index'] = 0
            except Exception as e:
                st.error(f"Error processing query: {e}")
        else:
            st.error("Please enter a query.")

# Retrieve patient data from session state
patients_data = st.session_state.get('patients_data', [])

if isinstance(patients_data, list) and len(patients_data) > 0:
    # Sidebar navigation for patients
    patient_ids = [p.get("Patient ID", "Unknown ID") for p in patients_data if isinstance(p, dict)]
    selected_patient_index = st.sidebar.radio("Select Patient", range(len(patient_ids)), 
                                               format_func=lambda i: f"Patient {patient_ids[i]}")
    
    selected_patient = patients_data[selected_patient_index]

    # Main panel to display patient data
    st.markdown("<h2 style='font-size:20px; font-weight:bold;'>Patient Details</h2>", unsafe_allow_html=True)
    
    # Display first three keys in a compact format
    patient_id = selected_patient.get("Patient ID", "Unknown")
    gender = selected_patient.get("Gender", "Unknown")
    age = selected_patient.get("Age", "Unknown")
    
    # Adjust font size and style of the first three keys
    st.markdown(f"""
        <div style="font-size:18px; font-weight:bold; margin-bottom:10px;">
            Patient ID: <span style="color:green;">{str(patient_id)}</span><br>
            Gender: {gender}<br>
            Age: <span style="color:green;">{str(age)}</span>
        </div>
    """, unsafe_allow_html=True)

    # Display remaining data
    for key, value in selected_patient.items():
        # Skip displaying the first three keys again
        if key in ["Patient ID", "Gender", "Age"]:
            continue

        # Font size and spacing for all keys
        st.markdown(f"<h3 style='font-size:18px; font-weight:bold; margin-bottom:0px;'>{key}:</h3>", unsafe_allow_html=True)

        # Handle list of dictionaries (e.g., Potential Risks)
        if isinstance(value, list) and all(isinstance(item, dict) for item in value):
            for index, item in enumerate(value, start=1):
                st.markdown(f"#### {key} {index}:", unsafe_allow_html=True)
                for sub_key, sub_value in item.items():
                    st.markdown(f"- **{sub_key}:** {sub_value}", unsafe_allow_html=True)

        # Handle simple lists (e.g., Recommended Next Steps)
        elif isinstance(value, list):
            for item in value:
                st.markdown(f"- {item}", unsafe_allow_html=True)
            st.markdown("<br>", unsafe_allow_html=True)

        # Handle dictionaries
        elif isinstance(value, dict):
            for sub_key, sub_value in value.items():
                st.markdown(f"- **{sub_key}:** {sub_value}", unsafe_allow_html=True)

        # Handle scalar values (e.g., plain strings or numbers)
        else:
            st.markdown(f"<p style='font-size:16px; margin-bottom:0px;'>{value}</p>", unsafe_allow_html=True)
else:
    st.info("No valid data to display. Please enter a query.")
''')



In [125]:
import threading
import subprocess
import os
import signal

def terminate_existing_process():
    """
    Terminates any running Streamlit process on the specified port.
    """
    port = "8501"
    try:
        # Find processes using the specified port
        result = subprocess.run(
            ["lsof", "-i", f":{port}"], stdout=subprocess.PIPE, text=True
        )
        lines = result.stdout.splitlines()
        
        for line in lines:
            if "LISTEN" in line:
                parts = line.split()
                pid = int(parts[1])  # The second column contains the process ID
                os.kill(pid, signal.SIGKILL)  # Terminate the process
                print(f"Terminated process with PID {pid} on port {port}")
    except Exception as e:
        print(f"Error terminating process: {e}")

def run_app():
    """
    Runs the Streamlit app on the specified port.
    """
    terminate_existing_process()  # Ensure no old processes are running
    process = subprocess.Popen(
        ['streamlit', 'run', 'plan3.py', '--server.port', '8501'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    for line in process.stdout:
        print(line, end='')

threading.Thread(target=run_app).start()

Terminated process with PID 21122 on port 8501
Terminated process with PID 21122 on port 8501

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.145.11.157:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
